In [1]:
from calendar import month
from operator import index
import snscrape.modules.twitter as sntwitter
import pandas as pd
import glob
import datetime

release_dates = pd.read_csv("Release dates - data.csv")
release_dates_map = {}

folderpathrt = "rotten_info/*"
filesrt = []

for file in glob.glob(folderpathrt):
    filesrt.append(file)

for file in filesrt:
    print(file)
    movies = pd.read_csv(file)

    tweets = []
    for ind, movie in movies.iterrows():
        movien = movie['Release Group']
        if not pd.isna(movie['Release Date (Theaters)']):
            release_date = ('').join(movie['Release Date (Theaters)'].strip('[]').split(',')).replace("'", "")
            rd = datetime.datetime.strptime(release_date, "%b %d %Y")
            # if release_dates_map.get(movien.lower(), ''):
            release_dates_map[movien.lower()] = rd

folderpath = "Box Office Mojo/*"
files = []

for file in glob.glob(folderpath):
    files.append(file)

for ind, row in release_dates.iterrows():
    if row['Release-Date']:
        rd = datetime.datetime.strptime(row['Release-Date'], "%Y-%m-%d")
        if not release_dates_map.get(row['Movie'].lower(), None):
            release_dates_map[row['Movie'].lower()] = rd
    
count = 0
c = 0
for file in files:
    if '2016' in file:
        print(file)
        movies = pd.read_csv(file)

        tweets = []
        for ind, movie in movies.iterrows():
            c += 1
            query_txt = movie['Release Group']
            release_date = release_dates_map.get(movie['Release Group'].lower(), None)
            if release_date:
                count += 1
                # rd = datetime.datetime.strptime(release_date, "%Y-%m-%d")
                since = release_date - datetime.timedelta(weeks=4)
                until = release_date + datetime.timedelta(weeks=1)
                query_txt += " since:%s until:%s" % (since.date(), until.date())
                print(movie['Release Group'], release_date.date())

                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query_txt).get_items()):
                    if i>5000:
                        break
                    tweets.append([movie['Release Group'], tweet.date, tweet.id, tweet.content, tweet.user.username])

        tweets_df = pd.DataFrame(tweets, columns=['Movie Name', 'Datetime', 'Tweet Id', 'Text', 'Username'])
        tweets_df.to_csv("Twitter/tweets_%s" % file.split('/')[1], index=False, escapechar=r'|')
        break


Box Office Mojo/2016.csv
Captain America: Civil War 2016-05-06
Rogue One: A Star Wars Story 2016-12-16
Finding Dory 2016-06-17
Zootopia 2016-03-04
The Jungle Book 2016-04-15
The Secret Life of Pets 2016-07-08
Batman v Superman: Dawn of Justice 2016-03-25
Fantastic Beasts and Where to Find Them 2016-11-18
Deadpool 2016-02-12
Suicide Squad 2016-08-05
Doctor Strange 2016-11-04
Moana 2016-11-23
Sing 2016-12-21
The Mermaid 2016-02-19
X-Men: Apocalypse 2016-05-27
Kung Fu Panda 3 2016-01-29
La La Land 2016-12-09
Warcraft 2016-06-10
Jason Bourne 2016-07-29
Ice Age 5: Collision Course 2016-07-22
Independence Day: Resurgence 2016-06-24
Your Name. 2017-04-07
The Legend of Tarzan 2016-07-01
The Angry Birds Movie 2016-05-20
Trolls 2016-11-04
Star Trek Beyond 2016-07-22
The Great Wall 2017-02-17
Now You See Me 2 2016-06-10
The Conjuring 2 2016-06-10
Resident Evil: The Final Chapter 2017-01-27
Dangal 2016-12-21
Passengers 2016-12-21
Alice Through the Looking Glass 2016-05-27
Miss Peregrine's Home for